Forecast sales using store, promotion, and competitor data
==========================================================

Rossmann operates over $3,000$ drug stores in $7$ European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

In their first Kaggle competition, Rossmann is challenging you to predict $6$ weeks of daily sales for $1,115$ stores located across Germany. Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation. By helping Rossmann create a robust prediction model, you will help store managers stay focused on what’s most important to them: their customers and their teams! 

You are provided with historical sales data for $1,115$ Rossmann stores. The task is to forecast the *Sales* column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

Files

- *train.csv* - historical data including Sales
- *test.csv* - historical data excluding Sales
- *sample_submission.csv* - a sample submission file in the correct format
- *store.csv* - supplemental information about the stores

Data fields

Most of the fields are self-explanatory. The following are descriptions for those that aren't.

- *Id* - an Id that represents a (Store, Date) duple within the test set
- *Store* - a unique Id for each store
- *Sales* - the turnover for any given day (this is what you are predicting)
- *Customers* - the number of customers on a given day
- *Open* - an indicator for whether the store was open: 0 = closed, 1 = open
- *StateHoliday* - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
- *SchoolHoliday* - indicates if the (Store, Date) was affected by the closure of public schools
- *StoreType* - differentiates between 4 different store models: a, b, c, d
- *Assortment* - describes an assortment level: a = basic, b = extra, c = extended
- *CompetitionDistance* - distance in meters to the nearest competitor store
- *CompetitionOpenSince*[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
- *Promo* - indicates whether a store is running a promo on that day
- *Promo2* - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- *Promo2Since*[Year/Week] - describes the year and calendar week when the store started participating in Promo2
- *PromoInterval* - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

Submissions are evaluated on the Root Mean Square Percentage Error (*RMSPE*). The RMSPE is calculated as

$\mbox{R} = \sqrt{\sum_{i=1}^{n}\frac{1}{n} \big{(}\frac{y_i-\hat{y_i}}{y_i}\big{)}^2}$

where $y_i$ denotes the sales of a single store on a single day and $\hat{y_i}$ denotes the corresponding prediction. Any day and store with $0$ sales is ignored in scoring.

Submission File

The file should contain a header and have the following format:


Id,Sales

1,0
2,0
3,0
etc.


In [3]:
import pandas as pd
import numpy as np
from numpy import dtype
store=pd.read_csv('store.csv', dtype={'Store': np.int64, 'StoreType': np.object, 'Assortment': np.object, 
                                     'CompetitionDistance':np.float32, 'CompetitionOpenSinceMonth':np.float16,
                                     'CompetitionOpenSinceYear': np.float16, 'Promo2': np.bool, 
                                      'Promo2SinceWeek': np.float16, 'Promo2SinceYear': np.object, 
                                      'PromoInterval': np.object},
                 na_values={''})

In [47]:
train=pd.read_csv('train.csv', dtype={'Store':np.int64, 'DayOfWeek':np.int8, 'Date': np.object, 
                                      'Sales':np.int64, 'Customers':np.int64, 'Open':np.bool, 
                                      'Promo': np.bool, 'StateHoliday':np.str, 'SchoolHoliday': np.bool},
                 parse_dates=[2])

In [48]:
test=pd.read_csv('test.csv', dtype={'Id':np.int64, 'Store': np.int64, 'DayOfWeek': np.int8, 'Date': np.object,
                                   'Promo': np.bool, 'StateHoliday': np.str, 'SchoolHoliday': np.bool, 
                                    'Open': np.float16},
                parse_dates=[3])

see store information

In [6]:
store.head(n=3)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,False,NaN,NaN,NaN
1,2,a,a,570,11,2007,True,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,True,14,2011,"Jan,Apr,Jul,Oct"


see train infromation

In [7]:
train.head(n=3)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,True,True,0,True
1,2,5,2015-07-31,6064,625,True,True,0,True
2,3,5,2015-07-31,8314,821,True,True,0,True


see test information

In [8]:
test.head(n=3)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1,True,0,False
1,2,3,4,2015-09-17,1,True,0,False
2,3,7,4,2015-09-17,1,True,0,False


data cleansing: remove null elements from Open

In [9]:
test.loc[ test.Open.isnull(), 'Open' ] = 1

In [10]:
train = train.loc[train.Sales > 0]

1. Initial solution is to use the mean value grouped by store, day of week and promo sale values:

In [61]:
group_on=[ 'Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday','SchoolHoliday']
means = train.groupby(group_on)['Sales'].mean()
# reset index to get a dataframe with n colums
means = means.reset_index()
# Merge with test dataframe to get sales predictions
test_merged = pd.merge(test, means, on = group_on, how='left')
# check if we have null values
test_merged.Sales.isnull().value_counts()
# use the sales mean for those
test_merged.fillna(test_merged.Sales.mean(), inplace=True)
# save merged to file
test_merged[[ 'Id', 'Sales' ]].to_csv( 'result', index = False )
print(test_merged.Sales.mean())

6170.32630429


In [60]:
from sklearn.preprocessing import LabelEncoder 
s_train = train 
s_train.StateHoliday = LabelEncoder().fit_transform(s_train.StateHoliday)
s_train.SchoolHoliday = LabelEncoder().fit_transform(s_train.SchoolHoliday)
s_train.Promo = LabelEncoder().fit_transform(s_train.Promo)
s_train.Open = LabelEncoder().fit_transform(s_train.Open)
s_train.Promo = LabelEncoder().fit_transform(s_train.Promo)
s_train.DayOfWeek = LabelEncoder().fit_transform(s_train.DayOfWeek)
s_train.Date = LabelEncoder().fit_transform(s_train.Date.apply(lambda x: x.timetuple().tm_yday))
s_train.Store = LabelEncoder().fit_transform(s_train.Store)

s_test = test 
s_test.StateHoliday = LabelEncoder().fit_transform(s_test.StateHoliday)
s_test.SchoolHoliday = LabelEncoder().fit_transform(s_test.SchoolHoliday)
s_test.Promo = LabelEncoder().fit_transform(s_test.Promo)
s_test.Open = LabelEncoder().fit_transform(s_test.Open)
s_test.Promo = LabelEncoder().fit_transform(s_test.Promo)
s_test.DayOfWeek = LabelEncoder().fit_transform(s_test.DayOfWeek)
s_test.Date = LabelEncoder().fit_transform(s_test.Date.apply(lambda x: x.timetuple().tm_yday))
s_test.Store = LabelEncoder().fit_transform(s_test.Store)

AttributeError: 'numpy.int64' object has no attribute 'timetuple'

In [50]:
from sklearn.ensemble import RandomForestRegressor
cols = ['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday','SchoolHoliday']
rf = RandomForestRegressor(n_estimators=20)
rf.fit(s_train[cols], s_train.Sales)
x=rf.predict(s_test[cols])

/usr/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/lib/p

In [68]:
rfd=pd.DataFrame(dict(Id=s_test.Id,Sales=x))
rfd.fillna(test_merged.Sales.mean(), inplace=True)
rfd.to_csv( 'result_rf', index = False )

In [69]:
(test_merged.Sales-x).mean()

112.34163460913078